In [1]:
#Importing the usual libraries.
import numpy as np
import tensorly as tl
import seaborn as sns
import multilinearalgebra 
import matplotlib.pyplot as plt

from scipy.io import loadmat

#Setting style options.
sns.set()
np.set_printoptions(3)

Using numpy backend.


The following packages are used in this notebook:

- `numpy 1.18.1`
- `tensorly 0.4.3`
- `seaborn 0.10.0`

# Problem 1

For a third-order tensor $\mathbf{\mathcal{X}} \in \mathbb{C}^{I \times J \times K}$ implement the truncated high-order singular value decomposition (HOSVD), using the following prototype function:

\begin{equation}
    [\mathbf{\mathcal{S}},\mathbf{U}^{(1)},\mathbf{U}^{(2)},\mathbf{U}^{(3)}] = hosvd(\mathbf{\mathcal{X}})
\end{equation}

Hint: Use the file “hosvd_test.mat” to validate your results.

<font color='red'>Solution:</font>

In [2]:
def HOSVD(tensor,*ranks):
    
    #Full-rank HOSVD
    if len(ranks) == 0:
        U = []
        for i in range(0,tensor.ndim):
            [u,_,_] = np.linalg.svd(multilinearalgebra.unfold(tensor,i))
            u = u.conj().T
            U.append(u)
        
        if len(np.array(U).shape) == 1:
            k = np.array(U)[-2:]
            U = np.append(k,(np.array(U)[0:-2:]))
        
        else:
            k = np.array(U)[-2:]
            U = np.append(k,(np.array(U)[0:-2:])).reshape(np.array(U).shape)
        
        S = multilinearalgebra.ten_mat_multiprod(tensor,np.array(U))
    
        for i in range(0,tensor.ndim):
            U[i] = U[i].conj().T
            
    #Truncated HOSVD       
    else:
        U = []
        for i in range(0,X.ndim):
            [u,s,_] = np.linalg.svd(multilinearalgebra.unfold(tensor,i))
            u = u[:,0:ranks[i]]
            u = u.conj().T
            U.append(u)
        
        if len(np.array(U).shape) == 1:
            k = np.array(U)[-2:]
            U = np.append(k,(np.array(U)[0:-2:]))
        
        else:
            k = np.array(U)[-2:]
            U = np.append(k,(np.array(U)[0:-2:])).reshape(np.array(U).shape)
        
        S = multilinearalgebra.ten_mat_multiprod(tensor,np.array(U))
        
        for i in range(0,tensor.ndim):
            U[i] = U[i].conj().T
    
    return S,U,S.shape

In [3]:
def HOSVD_epsilon(tensor,epsilon):
    
    U = []
    for i in range(0,tensor.ndim):
        [u,s,_] = np.linalg.svd(multilinearalgebra.unfold(tensor,i))
        s[s < epsilon] = 0
        s = [i for i in s if i != 0]
        s = len(s)
        u = u[:,0:s]
        u = u.conj().T
        U.append(u)
        
    if len(np.array(U).shape) == 1:
        k = np.array(U)[-2:]
        U = np.append(k,(np.array(U)[0:-2:]))
        
    else:
        k = np.array(U)[-2:]
        U = np.append(k,(np.array(U)[0:-2:])).reshape(np.array(U).shape)
        
    S = multilinearalgebra.ten_mat_multiprod(tensor,np.array(U))
    
    for i in range(0,tensor.ndim):
        U[i] = U[i].conj().T
    
    return S,U,S.shape

In [4]:
def HOOI(tensor,epsilon,*ranks):
    
    #Full-rank HOOI
    if len(ranks) == 0:
        [S_init,U_init,_] = HOSVD(tensor)
        
        for k in range(0,10):
            U = []
            
            for i in range(0,tensor.ndim):
                #Matrix Selection
                aux = np.ones(tensor.ndim, dtype=bool)
                aux[i] = False
                U_aux = np.asarray(U_init)
                U_aux = U_aux[aux]
            
                #Creating the list of U matrices
                modes = np.zeros([tensor.ndim-1],dtype='int8')
                for j in range(0,tensor.ndim-1):
                    modes[j] = multilinearalgebra.mode(U_aux[j],tensor)
                
                u = multilinearalgebra.ten_mat_multiprod(tensor,np.array(U_aux),*modes)
                [u,_,_] = np.linalg.svd(multilinearalgebra.unfold(u,i))
                u = u.conj().T
                U.append(u)
            
            #Creating the core tensor
            S = multilinearalgebra.ten_mat_multiprod(tensor,np.array(U),*np.arange(0,tensor.ndim))
            
            #Convergence
            if multilinearalgebra.normalized_mean_square_error(S,S_init) > epsilon:
                #print('NMSE Error for the iteration',k + 1,'.')
                #print(multilinearalgebra.normalized_mean_square_error(S,S_init))
                S_init = S
                U_init = U 
            
            else:
                for i in range(0,tensor.ndim):
                    U[i] = U[i].conj().T
                    
                break
                
    #Truncated HOOI          
    else:
        [S_init,U_init,_] = HOSVD(tensor,*ranks)
        
        for k in range(0,10):
            U = []
            
            for i in range(0,tensor.ndim):
                #Matrix Selection
                aux = np.ones(tensor.ndim, dtype=bool)
                aux[i] = False
                U_aux = np.asarray(U_init)
                U_aux = U_aux[aux]
            
                #Creating the list of U matrices
                modes = np.zeros([tensor.ndim-1],dtype='int8')
                for j in range(0,tensor.ndim-1):
                    modes[j] = multilinearalgebra.mode(U_aux[j],tensor)
                
                u = multilinearalgebra.ten_mat_multiprod(tensor,np.array(U_aux),*modes)
                [u,_,_] = np.linalg.svd(multilinearalgebra.unfold(u,i))
                u = u[:, :ranks[i]]
                u = u.conj().T
                U.append(u)
        
            #Creating the core tensor
            S = multilinearalgebra.ten_mat_multiprod(tensor,np.array(U),*np.arange(0,tensor.ndim))
            
            #Convergence
            if multilinearalgebra.normalized_mean_square_error(S,S_init) > epsilon:
                S_init = S
                U_init = U 
            
            else:
                for i in range(0,tensor.ndim):
                    U[i] = U[i].conj().T
                break
                
    k = np.array(U)[-2:]
    U = np.append(k,(np.array(U)[0:-2:]))

    return S,U,S.shape

# Test with a random tensor for HOSVD algorithm

In [5]:
X = np.random.randn(3,4,5)
[X_approx,U,R] = HOSVD(X)
X_approx = multilinearalgebra.ten_mat_multiprod(X_approx,np.array(U))

print('NMSE for full rank HOSVD:')
print(multilinearalgebra.normalized_mean_square_error(X_approx,X))
print('Multilinear rank for full rank HOSVD:')
print(R)

NMSE for full rank HOSVD:
3.4518648615703476e-31
Multilinear rank for full rank HOSVD:
(3, 4, 5)


# Test with hosvd_test.mat file for HOSVD algorithm

In [6]:
HOSVD_MAT = loadmat('hosvd_test.mat')
tenX = HOSVD_MAT['tenX'] 
tenS = HOSVD_MAT['tenS'] 
U1 = HOSVD_MAT['U1'] 
U2 = HOSVD_MAT['U2'] 
U3 = HOSVD_MAT['U3'] 
tenX = tenX.transpose(2, 0, 1)
tenS = tenS.transpose(2, 0, 1)

In [7]:
[S,U,R] = HOSVD(tenX)
tenX_approx = multilinearalgebra.ten_mat_multiprod(S,np.array(U))

In [8]:
print('NMSE for Stimation of U1:')
print(multilinearalgebra.normalized_mean_square_error(U[0],U1))

print('NMSE for Stimation of U2:')
print(multilinearalgebra.normalized_mean_square_error(U[1],U2))
 
print('NMSE for Stimation of U3:')
print(multilinearalgebra.normalized_mean_square_error(U[2],U3))

print('NMSE for Stimation of S:')
print(multilinearalgebra.normalized_mean_square_error(S,tenS))

print('NMSE for Stimation of X:')
print(multilinearalgebra.normalized_mean_square_error(tenX_approx,tenX))

print('Multilinear rank for full rank HOSVD:')
print(R)

NMSE for Stimation of U1:
0.0
NMSE for Stimation of U2:
0.0005771204134077601
NMSE for Stimation of U3:
0.024065389832155504
NMSE for Stimation of S:
1.5607066827251084
NMSE for Stimation of X:
3.079954617776256e-31
Multilinear rank for full rank HOSVD:
(5, 3, 4)


# Test with hosvd_test.mat file for HOOI algorithm

In [9]:
[S,U,R] = HOOI(tenX,10**-30)
tenX_approx = multilinearalgebra.ten_mat_multiprod(S,np.array(U))

In [10]:
print('NMSE for Stimation of U1:')
print(multilinearalgebra.normalized_mean_square_error(U[0],U1))

print('NMSE for Stimation of U2:')
print(multilinearalgebra.normalized_mean_square_error(U[1],U2))
 
print('NMSE for Stimation of U3:')
print(multilinearalgebra.normalized_mean_square_error(U[2],U3))

print('NMSE for Stimation of S:')
print(multilinearalgebra.normalized_mean_square_error(S,tenS))

print('NMSE for Stimation of X:')
print(multilinearalgebra.normalized_mean_square_error(tenX_approx,tenX))

print('Multilinear rank for full rank HOOI:')
print(R)

NMSE for Stimation of U1:
2.9505246748012458e-31
NMSE for Stimation of U2:
0.02018512441411901
NMSE for Stimation of U3:
1.0086379790151503
NMSE for Stimation of S:
1.560706682725109
NMSE for Stimation of X:
2.310073340453022e-31
Multilinear rank for full rank HOOI:
(5, 3, 4)


# Problem 2

Consider the two third-order tensors  $\mathbf{\mathcal{X}} \in \mathbb{C}^{8 \times 4 \times 10}$ and $\mathbf{\mathcal{X}} \in \mathbb{C}^{5 \times 5 \times 5}$ provided in the data file “hosvd_denoising.mat”. By using your HOSVD prototype function, find a low multilinear rank approximation for these tensors, defined as $\mathbf{\mathcal{X}} \in \mathbb{C}^{R_{1} \times R_{2} \times R_{3}}$  and $\mathbf{\mathcal{Y}} \in \mathbb{C}^{P_{1} \times P_{2} \times P_{3}}$. Then, calculate the normalized mean square error (NMSE) between the original tensor and its approximation, i.e.,:

\begin{equation}
    NMSE(\mathbf{\mathcal{\tilde{X}}}) = \frac{||\mathbf{\mathcal{\tilde{X}}} - \mathbf{\mathcal{X}}||^{2}_{F}}{||\mathbf{\mathcal{X}}||^{2}_{F}}
\end{equation}

\begin{equation}
    NMSE(\mathbf{\mathcal{\tilde{Y}}}) = \frac{||\mathbf{\mathcal{\tilde{Y}}} - \mathbf{\mathcal{Y}}||^{2}_{F}}{||\mathbf{\mathcal{Y}}||^{2}_{F}}
\end{equation}

Hint: The multilinear ranks of X and Y can be found by analysing the profile of the 1-mode, 2-mode and 3-mode singular values of these tensors.

<font color='red'>Solution:</font>

# Test with a random tensor for HOSVD algorithm

In [11]:
X = np.random.randn(3,4,5)
[X_approx1,U1,R1] = HOSVD(X)
[X_approx2,U2,R2] = HOSVD(X,*[3,2,3])
[X_approx3,U3,R3] = HOSVD_epsilon(X,2)
X_approx1 = multilinearalgebra.ten_mat_multiprod(X_approx1,np.array(U1))
X_approx2 = multilinearalgebra.ten_mat_multiprod(X_approx2,np.array(U2))
X_approx3 = multilinearalgebra.ten_mat_multiprod(X_approx3,np.array(U3))

print('NMSE for full rank HOSVD:')
print(multilinearalgebra.normalized_mean_square_error(X_approx1,X))
print('NMSE for truncated HOSVD:')
print(multilinearalgebra.normalized_mean_square_error(X_approx2,X))
print('NMSE for truncated HOSVD by an epsion:')
print(multilinearalgebra.normalized_mean_square_error(X_approx3,X))

print('Multilinear Rank of Y for full rank HOSVD:')
print(R1)
print('Multilinear Rank of Y for truncated HOSVD1:')
print(R2)
print('Multilinear Rank of Y for truncated HOSVD2:')
print(R3)

NMSE for full rank HOSVD:
1.1660996401587113e-30
NMSE for truncated HOSVD:
0.43390703192649954
NMSE for truncated HOSVD by an epsion:
0.05130476035488607
Multilinear Rank of Y for full rank HOSVD:
(3, 4, 5)
Multilinear Rank of Y for truncated HOSVD1:
(3, 2, 3)
Multilinear Rank of Y for truncated HOSVD2:
(3, 4, 4)


# Test with hosvd_denoising.mat file for HOSVD algorithm

In [12]:
HOSVD_MAT = loadmat('hosvd_denoising.mat')
tenX_noise = HOSVD_MAT['tenX_noise'] 
tenY_noise = HOSVD_MAT['tenY_noise'] 
tenX_noise = tenX_noise.transpose(2, 0, 1)
tenY_noise = tenY_noise.transpose(2, 0, 1)

In [13]:
[S1,U1,R1] = HOSVD(tenX_noise)
[S2,U2,R2] = HOSVD(tenX_noise,*[5,4,2])
[S3,U3,R3] = HOSVD_epsilon(tenX_noise,3)
tenX_noise_approx1 = multilinearalgebra.ten_mat_multiprod(S1,np.array(U1))
tenX_noise_approx2 = multilinearalgebra.ten_mat_multiprod(S2,np.array(U2))
tenX_noise_approx3 = multilinearalgebra.ten_mat_multiprod(S3,np.array(U3))

print('NMSE for full rank HOSVD:')
print(multilinearalgebra.normalized_mean_square_error(tenX_noise_approx1,tenX_noise))
print('NMSE for truncated HOSVD:')
print(multilinearalgebra.normalized_mean_square_error(tenX_noise_approx2,tenX_noise))
print('NMSE for truncated HOSVD2 by an epsilon:')
print(multilinearalgebra.normalized_mean_square_error(tenX_noise_approx3,tenX_noise))

print('Multilinear Rank of X for full rank HOSVD:')
print(R1)
print('Multilinear Rank of X for truncated HOSVD:')
print(R2)
print('Multilinear Rank of X for truncated HOSVD by an epsilon:')
print(R3)

NMSE for full rank HOSVD:
5.103623423476849e-31
NMSE for truncated HOSVD:
0.047986502757933126
NMSE for truncated HOSVD2 by an epsilon:
0.00012141882449270083
Multilinear Rank of X for full rank HOSVD:
(10, 8, 4)
Multilinear Rank of X for truncated HOSVD:
(5, 4, 2)
Multilinear Rank of X for truncated HOSVD by an epsilon:
(5, 2, 3)


In [14]:
[S1,U1,R1] = HOSVD(tenY_noise)
[S2,U2,R2] = HOSVD(tenY_noise,*[4,4,2])
[S3,U3,R3] = HOSVD_epsilon(tenY_noise,0.5)
tenY_noise_approx1 = multilinearalgebra.ten_mat_multiprod(S1,np.array(U1))
tenY_noise_approx2 = multilinearalgebra.ten_mat_multiprod(S2,np.array(U2))
tenY_noise_approx3 = multilinearalgebra.ten_mat_multiprod(S3,np.array(U3))

print('NMSE for full rank HOSVD:')
print(multilinearalgebra.normalized_mean_square_error(tenY_noise_approx1,tenY_noise))
print('NMSE for truncated HOSVD1:')
print(multilinearalgebra.normalized_mean_square_error(tenY_noise_approx2,tenY_noise))
print('NMSE for truncated HOSVD2:')
print(multilinearalgebra.normalized_mean_square_error(tenY_noise_approx3,tenY_noise))

print('Multilinear Rank of Y for full rank HOSVD:')
print(R1)
print('Multilinear Rank of Y for truncated HOSVD:')
print(R2)
print('Multilinear Rank of Y for truncated HOSVD by an epsilon:')
print(R3)

NMSE for full rank HOSVD:
1.2820802734839317e-31
NMSE for truncated HOSVD1:
0.012793078937238243
NMSE for truncated HOSVD2:
0.01758259280933224
Multilinear Rank of Y for full rank HOSVD:
(5, 5, 5)
Multilinear Rank of Y for truncated HOSVD:
(4, 4, 2)
Multilinear Rank of Y for truncated HOSVD by an epsilon:
(3, 2, 2)


## About this notebook

**Author**: Kenneth B. dos A. Benício

**Uptaded on**: 2020-04-30